In [10]:
import logging
import pandas as pd
import numpy as np
import scanpy as sc
import os
import sys

current_folder = str(globals()['_dh'][0])
ROOT_PATH = current_folder.split('src/')[0]

os.chdir(ROOT_PATH)
sys.path.append(ROOT_PATH + 'src')

import utils.basics as uf
current_folder = str(globals()['_dh'][0])
ROOT_PATH = current_folder.split('src/')[0]
os.chdir(ROOT_PATH)
# Total 80411 disease-gene triples
np.random.seed(2022)

# ! Load
state_df =  uf.pickle_load(f'data/static/static_data.pkl')
risk_level = uf.pickle_load(f'data/dynamic/dynamic_data_trend.pkl')
data_trend =  uf.pickle_load(f'data/dynamic/dynamic_data_risk_level.pkl')


In [14]:
fips_to_state = dict(zip(risk_level['fips'],risk_level['state_name']))
state_df['state_name'] = state_df.fips.map(fips_to_state)
new_col_order = ['state_name'] + [col for col in state_df if col != 'state_name']
state_df = state_df[new_col_order]
state_df

,state_name,fips,Population,under_20,over_65,White,Black,Multiple_race,Not_Hispanic,Hispanic,...,healthcare_utilization,poor_health_status,adults_at_high_risk,poverty_rate,social_vulnerability_index,Healthcare Access and Quality Index,Older_at_high_risk,dem_percent,rep_percent,other_percent
0,Alabama,01,4903185,0.248091,0.173323,0.691264,0.267844,0.017718,0.954463,0.045537,...,0.181,0.227,0.43,0.154,0.614136,83.4,0.510,0.366,0.620,0.014
1,Alaska,02,731545,0.269091,0.125198,0.652712,0.037056,0.074772,0.927261,0.072739,...,0.135,0.155,0.33,0.107,0.458578,88.4,0.494,0.428,0.528,0.044
2,Arizona,04,7278717,0.252599,0.179789,0.826168,0.051794,0.029330,0.682555,0.317445,...,0.139,0.19,0.39,0.134,0.720161,88.0,0.591,0.494,0.491,0.016
3,Arkansas,05,3017804,0.258111,0.173597,0.790395,0.156752,0.022066,0.921588,0.078412,...,0.157,0.252,0.44,0.165,0.650340,83.8,0.503,0.348,0.624,0.028
4,California,06,39512223,0.251198,0.147755,0.719391,0.064607,0.039839,0.605821,0.394179,...,0.119,0.182,0.33,0.118,0.654184,91.0,0.560,0.635,0.343,0.022
5,Colorado,08,5758736,0.244493,0.146284,0.869394,0.045927,0.031373,0.781740,0.218260,...,0.121,0.14,0.31,0.094,0.329029,90.8,0.591,0.554,0.419,0.027
6,Connecticut,09,3565287,0.232683,0.176772,0.797002,0.121893,0.024721,0.831443,0.168557,...,0.099,0.149,0.36,0.102,0.425818,91.5,0.599,0.593,0.392,0.016
7,Delaware,10,973764,0.235080,0.193996,0.692237,0.231621,0.027377,0.904093,0.095907,...,0.106,0.187,0.41,0.109,0.447416,88.6,0.583,0.587,0.398,0.015
8,Florida,12,21477737,0.219510,0.209395,0.772772,0.169176,0.022275,0.736292,0.263708,...,0.16,0.197,0.42,0.127,0.605656,89.6,0.612,0.479,0.512,0.009
9,Georgia,13,10617423,0.263877,0.142874,0.601983,0.325705,0.022170,0.901226,0.098774,...,0.178,0.2,0.36,0.134,0.568470,85.5,0.509,0.495,0.493,0.012


In [2]:
risk_level

,state_name,fips,Week,Week_start,Week_end,hospitalization_per_100k,reported_cases_per_100k,previous_infection_12w,Dose1_Pop_Pct,Series_Complete_Pop_Pct,...,H1_Public information campaigns,H2_Testing policy,H3_Contact tracing,H6M_Facial Coverings,H8M_Protection of elderly people,Trend,Trend_1w,Trend_2w,Trend_3w,Trend_4w
0,Alabama,01,2020-12-14/2020-12-20,2020-12-14,2020-12-20,54.9,547.0,3.5,0.0,0.0,...,coordinated public information campaign (eg ac...,"open public testing (eg ""drive through"" testin...",limited contact tracing; not done for all cases,Required in all shared/public spaces outside t...,"Narrow restrictions for isolation, hygiene in ...",Moderate Increasing,Moderate Increasing,Substantial increasing,Moderate Increasing,Stable
1,Alabama,01,2020-12-21/2020-12-27,2020-12-21,2020-12-27,58.1,472.7,3.8,0.0,0.0,...,coordinated public information campaign (eg ac...,"open public testing (eg ""drive through"" testin...",limited contact tracing; not done for all cases,Required in all shared/public spaces outside t...,"Narrow restrictions for isolation, hygiene in ...",Stable,Moderate Increasing,Moderate Increasing,Substantial increasing,Moderate Increasing
2,Alabama,01,2020-12-28/2021-01-03,2020-12-28,2021-01-03,61.2,536.5,4.2,0.0,0.0,...,coordinated public information campaign (eg ac...,"open public testing (eg ""drive through"" testin...",limited contact tracing; not done for all cases,Required in all shared/public spaces outside t...,"Narrow restrictions for isolation, hygiene in ...",Stable,Stable,Moderate Increasing,Moderate Increasing,Substantial increasing
3,Alabama,01,2021-01-04/2021-01-10,2021-01-04,2021-01-10,61.6,611.2,4.7,0.0,0.0,...,coordinated public information campaign (eg ac...,"open public testing (eg ""drive through"" testin...",limited contact tracing; not done for all cases,Required in all shared/public spaces outside t...,"Narrow restrictions for isolation, hygiene in ...",Stable,Stable,Stable,Moderate Increasing,Moderate Increasing
4,Alabama,01,2021-01-11/2021-01-17,2021-01-11,2021-01-17,55.1,422.1,4.9,0.0,0.0,...,coordinated public information campaign (eg ac...,"open public testing (eg ""drive through"" testin...",limited contact tracing; not done for all cases,Required in all shared/public spaces outside t...,"Narrow restrictions for isolation, hygiene in ...",Moderate Decrease,Stable,Stable,Stable,Moderate Increasing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5295,Wyoming,56,2022-11-21/2022-11-27,2022-11-21,2022-11-27,10.5,101.6,1.0,60.4,52.7,...,coordinated public information campaign (eg ac...,"open public testing (eg ""drive through"" testin...",limited contact tracing; not done for all cases,No policy,"Recommended isolation, hygiene, and visitor re...",Substantial increasing,Moderate Decrease,Stable,Substantial increasing,Moderate Decrease
5296,Wyoming,56,2022-11-28/2022-12-04,2022-11-28,2022-12-04,10.5,86.2,1.0,60.5,52.8,...,coordinated public information campaign (eg ac...,"open public testing (eg ""drive through"" testin...",limited contact tracing; not done for all cases,No policy,"Recommended isolation, hygiene, and visitor re...",Stable,Substantial increasing,Moderate Decrease,Stable,Substantial increasing
5297,Wyoming,56,2022-12-05/2022-12-11,2022-12-05,2022-12-11,8.1,133.0,1.0,60.5,52.8,...,coordinated public information campaign (eg ac...,"open public testing (eg ""drive through"" testin...",limited contact tracing; not done for all cases,No policy,"Recommended isolation, hygiene, and visitor re...",Substantial Decrease,Stable,Substantial increasing,Moderate Decrease,Stable
5298,Wyoming,56,2022-12-12/2022-12-18,2022-12-12,2022-12-18,6.0,86.9,1.0,60.6,52.8,...,coordinated public information campaign (eg ac...,"open public testing (eg ""drive through"" testin...",limited contact tracing; not done for all cases,No policy,"Recommended isolation, hygiene, and visitor re...",Substantial Decrease,Substantial Decrease,Stable,Substantial increasing,M

In [3]:
data_trend

,state_name,fips,Week,Week_start,Week_end,hospitalization_per_100k,reported_cases_per_100k,previous_infection_12w,Dose1_Pop_Pct,Series_Complete_Pop_Pct,...,H1_Public information campaigns,H2_Testing policy,H3_Contact tracing,H6M_Facial Coverings,H8M_Protection of elderly people,Risk_level,Risk_level_1w,Risk_level_2w,Risk_level_3w,Risk_level_4w
0,Alabama,01,2020-12-14/2020-12-20,2020-12-14,2020-12-20,54.9,547.0,3.5,0.0,0.0,...,coordinated public information campaign (eg ac...,"open public testing (eg ""drive through"" testin...",limited contact tracing; not done for all cases,Required in all shared/public spaces outside t...,"Narrow restrictions for isolation, hygiene in ...",High,High,High,High,High
1,Alabama,01,2020-12-21/2020-12-27,2020-12-21,2020-12-27,58.1,472.7,3.8,0.0,0.0,...,coordinated public information campaign (eg ac...,"open public testing (eg ""drive through"" testin...",limited contact tracing; not done for all cases,Required in all shared/public spaces outside t...,"Narrow restrictions for isolation, hygiene in ...",High,High,High,High,High
2,Alabama,01,2020-12-28/2021-01-03,2020-12-28,2021-01-03,61.2,536.5,4.2,0.0,0.0,...,coordinated public information campaign (eg ac...,"open public testing (eg ""drive through"" testin...",limited contact tracing; not done for all cases,Required in all shared/public spaces outside t...,"Narrow restrictions for isolation, hygiene in ...",High,High,High,High,High
3,Alabama,01,2021-01-04/2021-01-10,2021-01-04,2021-01-10,61.6,611.2,4.7,0.0,0.0,...,coordinated public information campaign (eg ac...,"open public testing (eg ""drive through"" testin...",limited contact tracing; not done for all cases,Required in all shared/public spaces outside t...,"Narrow restrictions for isolation, hygiene in ...",High,High,High,High,High
4,Alabama,01,2021-01-11/2021-01-17,2021-01-11,2021-01-17,55.1,422.1,4.9,0.0,0.0,...,coordinated public information campaign (eg ac...,"open public testing (eg ""drive through"" testin...",limited contact tracing; not done for all cases,Required in all shared/public spaces outside t...,"Narrow restrictions for isolation, hygiene in ...",High,High,High,High,High
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5295,Wyoming,56,2022-11-21/2022-11-27,2022-11-21,2022-11-27,10.5,101.6,1.0,60.4,52.7,...,coordinated public information campaign (eg ac...,"open public testing (eg ""drive through"" testin...",limited contact tracing; not done for all cases,No policy,"Recommended isolation, hygiene, and visitor re...",medium,low,low,low,low
5296,Wyoming,56,2022-11-28/2022-12-04,2022-11-28,2022-12-04,10.5,86.2,1.0,60.5,52.8,...,coordinated public information campaign (eg ac...,"open public testing (eg ""drive through"" testin...",limited contact tracing; not done for all cases,No policy,"Recommended isolation, hygiene, and visitor re...",medium,medium,low,low,low
5297,Wyoming,56,2022-12-05/2022-12-11,2022-12-05,2022-12-11,8.1,133.0,1.0,60.5,52.8,...,coordinated public information campaign (eg ac...,"open public testing (eg ""drive through"" testin...",limited contact tracing; not done for all cases,No policy,"Recommended isolation, hygiene, and visitor re...",low,medium,medium,low,low
5298,Wyoming,56,2022-12-12/2022-12-18,2022-12-12,2022-12-18,6.0,86.9,1.0,60.6,52.8,...,coordinated public information campaign (eg ac...,"open public testing (eg ""drive through"" testin...",limited contact tracing; not done for all cases,No policy,"Recommended isolation, hygiene, and visitor re...",low,low,medium,medium,low
